In [51]:
#!/usr/bin/python

In [52]:
!git clone https://github.com/st1800eafit/st1800_20211.git
!pip install nltk
!pip install stop-words
!pip install pandas
!pip install scikit-learn

fatal: destination path 'st1800_20211' already exists and is not an empty directory.


In [53]:
import os
import glob
import re 
import nltk 
import pandas as pd
import pickle

In [54]:
files_location = os.path.join("/","content","st1800_20211","datasets", "papers_sample_pdf/")
output_file_location = os.path.join("/","content/")
output_file = "scikit_model_vectorized.sav"

filenames = glob.glob(files_location+"*.txt")
corpus_per_document = []

for file in filenames:
  doc_corpus = open(file, "r").read()
  corpus_per_document.append(doc_corpus)

# Adicionamos el XML sin tags
filename_xml = glob.glob(files_location+"*.xml")
xml_file = open (filename_xml[0], "r").read()
xml_file = re.sub('<[^>]*>', "", xml_file)
corpus_per_document.append(xml_file)

In [55]:
# Luego aplicamos de la libreria de Scikit-Learn la tecnica de TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Excluimos las palabras menores a 2 y que esten en mas de 50 porciento de los textos, adicionalmente incluimos bigrams
tfidf = TfidfVectorizer(min_df=2, max_df=0.5, ngram_range=(1, 2))
features = tfidf.fit_transform(corpus_per_document)
pd.DataFrame(
    features.todense(),
    columns=tfidf.get_feature_names()
)

,00,000,003,007,01 007,01 01,01 02,01 10,02,02 01,02 03,025,027,03,03 02,03 04,04,04 03,04 05,04 09,05,05 04,05 06,06 07,07,07 08,07 09,0704,0710,08,08 09,08 11,0803,0808,0811,0812,09,09 10,09 11,10 10,...,yields the,yn,york ny,yu,yx,z0,zahl,zero as,zero entries,zero error,zero for,zero in,zero mean,zero probability,zero this,zhao,zheng,zi,ziegler,zn,zu,µi,µi we,µi µi,αi,δk,ηn,θ0,θ0 θ1,θ1,θj,θj with,λ1,λ2,λj,σn,τn,φi,φi φj,φj
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005010,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.015031,0.000000,0.000000,0.000000,0.145298,0.000000,0.000000,0.000000,0.008713,0.00501,0.00501,0.010021,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.005010,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015031,0.000000,0.005010,0.005010,0.005010
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.009056,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.009056,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.018113,0.009056,0.081508,0.009056,0.009056,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.006957,0.009056,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.003061,0.003061,0.000000,0.000000,0.000000,0.000000,0.006123,0.003521,0.005410,0.000000,0.006123,0.000000,0.000000,0.006123,0.000000,0.007042,0.006123,0.000000,0.003061,0.003521,0.018368,0.000000,0.021126,0.021126,0.021126,0.017605,0.003521,0.000000,0.000000,0.017605,0.007042,0.007042,0.000000,0.000000,0.003061,0.000000,0.012246,0.007042,0.003521,0.003521,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003521,0.002705,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003521,0.000000,0.003521,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002705,0.003521,0.003521,0.000000,0.003521,0.000000,0.000000,0.000000
3,0.011576,0.008682,0.000000,0.003328,0.003328,0.000000,0.000000,0.000000,0.002557,0.000000,0.000000,0.003328,0.003328,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.002894,0.000000,0.002894,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.006657,0.000000,0.003328,0.033285,0.000000,0.002557,0.003328,0.000000,0.00000,0.00000,0.000000,0.003328,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00

In [56]:
# Podremos aplicar otras tecnicas para representar en vectores los elementos del corpus. En este caso implementaremos Count Vectorizer
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

vector_docs = vectorizer.fit_transform(corpus_per_document)

# Con esta linea se puede observar la matriz de ocurrencias de cada palabra en los documentos
vector_docs.toarray()

# Con esta linea se pueden ver las palabras mas ocurrentes reconocidas por el CountVectorizer
pd.DataFrame(
    vector_docs.todense(),
    columns=vectorizer.get_feature_names()
)

,00,000,00005,00018732,0009,001,0010117,00112,00135,00144,00147,00158,0016,00169,00199,002,0020,00207,00207160,00207179,00215,00217,00222,00244,00245,0025,00272,0028,00290,00294527,003,0030,00315,00327,00347,00366,0037,00398,004,00433,...,σ12,σ2,σ22,σi,σi2,σn,σn1,σn2,σz2,τ1,τ2,τ24,τ3,τ4,τ8,τi,τj,τn,τδ,φ0,φ1,φ2,φe0,φi,φj,φm,φmax,φq,φs,φx,φy,φz,ψi,ψihψ,ψik2,ω2,ωi,ωm2,ℓ2,ℓn
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,6,1,2,5,1,3,1,6,3,0,0,0,0,0,0,0,0,0,0,0,5,4,0,1,1,12,8,5,1,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,1,1,2,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
3,4,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,56,0,0,35,4,8,0,0,0,0,10,4,3,35,1,1,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,147,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16
7,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,6,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0
8,0,4,1,1,1,8,0,2,2,2,2,2,3,2,2,6,1,2,2,1,2,2,2,2,2,1,2,1,2,1,10,1,2,2,2,2,1,2,5,2,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [57]:
# Tambien podemos aplicar Count Vectorizer (TF) pero utilizando bigramas y no solo la palabra (monograma)

# En este caso solo nos importaria el rango de los ngrams (maximo 2, minimo 1)
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), min_df=1)
vector_docs_bigram = bigram_vectorizer.fit_transform(corpus_per_document).toarray()
vector_docs_bigram

array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 1,  1,  0, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ..., 10,  3,  1],
       [ 2,  0,  1, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]])

In [58]:
# Finalmente podremos guardar la informacion para tenerla a la mano cuando se vaya a utilizar un modelo. Para esto utilizaremos pickle

# Guardar el modelo (En este caso el modelo de count vectorizer en bigramas)
output_filename = output_file_location+output_file
print(output_filename)
pickle.dump(bigram_vectorizer, open(output_filename, 'wb'))

# Importar el modelo para el futuro e imprimios los tokens reconocidos
loaded_model = pickle.load(open(output_filename, 'rb'))
print(loaded_model.get_feature_names())

/content/scikit_model_vectorized.sav
['00', '00 01', '00 14', '00 25', '00 31', '00 ek', '00 gt', '000', '000 101', '000 attributes', '000 confessions', '000 documents', '000 features', '000 repetitions', '000 runs', '000 times', '00005', '00005 oai', '00018732', '00018732 2016', '0009', '0009 oai', '001', '001 oai', '0010117', '0010117 arxiv', '00112', '00112 2018', '00112 proceedings', '00135', '00135 2018', '00135 entropy', '00144', '00144 2018', '00144 oai', '00147', '00147 2018', '00147 oai', '00158', '00158 2018', '00158 oai', '0016', '0016 oai', '00169', '00169 2018', '00169 oai', '00199', '00199 2018', '00199 oai', '002', '002 oai', '0020', '0020 oai', '00207', '00207 2018', '00207 ieee', '00207160', '00207160 2014', '00207160 2017', '00207179', '00207179 2016', '00215', '00215 2018', '00215 oai', '00217', '00217 2018', '00217 oai', '00222', '00222 2018', '00222 ieee', '00244', '00244 2018', '00244 doi', '00245', '00245 2018', '00245 doi', '0025', '0025 oai', '00272', '00272 20